In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "admin"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
   html.Div(
    id='hidden-div',
    children=[
        html.A(
            href="https://www.snhu.edu",
            children=[
                html.Img(
                    src='data:image/png;base64,{}'.format(encoded_image.decode()),
                    style={'width': '200px'}   
                )
            ]
        )
    ],
    style={'text-align': 'center'}
    ),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Center(html.H2('Project 2 - Jacob Ward')),
    html.Hr(),
    html.Div(
        
    dcc.RadioItems(
        id = 'filter-type',
        options = [
            'Water Rescue', 
            'Wilderness Rescue',
            'Individual Tracking', 
            'Reset'
        ], 
        value = 'Reset')
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         sort_action='native',
                         row_selectable = "single",
                         page_action='native',
                         page_size= 15,
                        ),
    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type == 'Water Rescue':
        query = {
            "animal_type":"Dog",
            "breed":{"$in":[
                "Labrador Retriever Mix",
                "Chesapeake Bay Retriever",
                "Newfoundland"
            ]}, 
            "sex_upon_outcome":"Intact Female",
            "age_upon_outcome_in_weeks":{"$gte":26},
            "age_upon_outcome_in_weeks":{"$lte":156}
        }
    elif filter_type == 'Wilderness Rescue':
        query = {
            
            
            "animal_type":"Dog",
            "breed":{"$in":[
                "German Shepherd", 
                "Alaskan Malamute", 
                "Old English Sheepdog", 
                "Siberian Husky",
                "Rottweiler"
            ]}, 
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte":26},
            "age_upon_outcome_in_weeks":{"$lte":156}
        }
    elif filter_type == 'Individual Tracking':
        query = {
            "animal_type":"Dog",
            "breed":{"$in":[
                "Doberman Pinscher", 
                "German Shepherd", 
                "Golden Retriever",
                "Bloodhound", 
                "Rottweiler"
            ]}, 
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte":20},
            "age_upon_outcome_in_weeks":{"$lte":300}
        }
    else:
        query = {}
    df = pd.DataFrame.from_records(db.read(query))
    df.drop(columns=['_id'], inplace=True)
    data = df.to_dict('records')
    
    return data

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('filter-type', 'value')])
def update_graphs(viewData, filter_type):
    if filter_type == 'Reset':
        filter_type = 'Adopting. (Filter has been reset.)'
    df = pd.DataFrame.from_dict(viewData)
    pie_chart = dcc.Graph(style={'width': '1000px', 'height': '500px'},         
           figure = px.pie(
               df,
               names='breed',
               title=f'Animals Best Suited For: {filter_type}'
           )
        )  
    pie_chart.figure.update_traces(textposition='inside', textinfo='percent+label')
    
    return [pie_chart]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    # this will work in conjunction with the update_map method for when selected_columns is None
    if not selected_columns:
        return []
    else:
        return [{
            'if': { 'column_id': i },
            'background_color': '#D2F3FF'
        } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):      
    dff = pd.DataFrame.from_dict(viewData)
    # To avoid errors when first loading and viewdata is None
    if viewData is None or len(viewData) == 0:
        return [html.Div("No available data quite yet.")]
    
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None or len(index) == 0:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]


# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server() 

Dash app running on https://mayorscarlet-earthfamous-3000.codio.io/proxy/8050/
